## アンサンブル学習
3種類のアンサンブル学習をスクラッチ実装していきます。そして、それぞれの効果を小さめのデータセットで確認します。

- ブレンディング
- バギング
- スタッキング

#### 小さなデータセットの用意
以前も利用した回帰のデータセットを用意します。

[House Prices: Advanced Regression Techniques](https://www.kaggle.com/c/house-prices-advanced-regression-techniques/data)

この中のtrain.csvをダウンロードし、目的変数としてSalePrice、説明変数として、GrLivAreaとYearBuiltを使います。

train.csvを学習用（train）8割、検証用（val）2割に分割してください。

#### scikit-learn
単一のモデルはスクラッチ実装ではなく、scikit-learnなどのライブラリの使用を推奨します。

[sklearn.linear_model.LinearRegression — scikit-learn 0.21.3 documentation](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LinearRegression.html)

[sklearn.svm.SVR — scikit-learn 0.21.3 documentation](https://scikit-learn.org/stable/modules/generated/sklearn.svm.SVR.html)

[sklearn.tree.DecisionTreeRegressor — scikit-learn 0.21.3 documentation](https://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeRegressor.html)

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split

df = pd.read_csv('train.csv')
df_select = df[['GrLivArea', 'YearBuilt', 'SalePrice']]
df_select.describe()

,GrLivArea,YearBuilt,SalePrice
count,1460.000000,1460.000000,1460.000000
mean,1515.463699,1971.267808,180921.195890
std,525.480383,30.202904,79442.502883
min,334.000000,1872.000000,34900.000000
25%,1129.500000,1954.000000,129975.000000
50%,1464.000000,1973.000000,163000.000000
75%,1776.750000,2000.000000,214000.000000
max,5642.000000,2010.000000,755000.000000


In [2]:
df_train = df_select.iloc[:int(len(df_select)*0.8), :]
df_val = df_select.iloc[int(len(df_select)*0.8):, :]
print(df_train.shape)
print(df_val.shape)

(1168, 3)
(292, 3)


In [3]:
X_train = df_train[['GrLivArea', 'YearBuilt']]
y_train = df_train["SalePrice"]
X_test = df_val[['GrLivArea', 'YearBuilt']]
y_test = df_val["SalePrice"]
X = df_select[['GrLivArea', 'YearBuilt']]
y = df_select["SalePrice"]

In [4]:
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet, LogisticRegression
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor

from sklearn.metrics import mean_squared_error

### 【問題1】ブレンディングのスクラッチ実装
ブレンディング をスクラッチ実装し、単一モデルより精度があがる例を 最低3つ 示してください。精度があがるとは、検証用データに対する平均二乗誤差（MSE）が小さくなることを指します。

#### ブレンディングとは
ブレンディングとは、N個の多様なモデルを独立して学習させ、推定結果を重み付けした上で足し合わせる方法です。最も単純には平均をとります。多様なモデルとは、以下のような条件を変化させることで作り出すものです。

- 手法（例：線形回帰、SVM、決定木、ニューラルネットワークなど）
- ハイパーパラメータ（例：SVMのカーネルの種類、重みの初期値など）
- 入力データの前処理の仕方（例：標準化、対数変換、PCAなど）

重要なのはそれぞれのモデルが大きく異なることです。

回帰問題でのブレンディングは非常に単純であるため、scikit-learnには用意されていません。

《補足》

分類問題の場合は、多数決を行います。回帰問題に比べると複雑なため、scikit-learnにはVotingClassifierが用意されています。

[sklearn.ensemble.VotingClassifier — scikit-learn 0.21.3 documentation](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.VotingClassifier.html)

In [5]:
class ScratchBlending():
    def __init__(self, models):
        self.models = models

# 不要だった
#     def fit(self, X, y):
#         for model in self.models:
#             model.fit(X, y)

    def predict(self, X):
        list_y_pred = []
        for model in self.models:
            list_y_pred.append(model.predict(X))
            
        y_pred = sum(list_y_pred)/len(list_y_pred)
        
        return y_pred

In [6]:
# パターン1
mse_single = []
lr_1 = LinearRegression()
lr_1.fit(X_train, y_train)
y_pred = lr_1.predict(X_test)
mse_single.append(mean_squared_error(y_pred, y_test))

svr_1 = SVR()
svr_1.fit(X_train, y_train)
y_pred = svr_1.predict(X_test)
mse_single.append(mean_squared_error(y_pred, y_test))

dt_1 = DecisionTreeRegressor()
dt_1.fit(X_train, y_train)
y_pred = dt_1.predict(X_test)
mse_single.append(mean_squared_error(y_pred, y_test))


models_1 = [lr_1, svr_1, dt_1]

blend_model = ScratchBlending(models_1)
y_pred = blend_model.predict(X_test)
mse = mean_squared_error(y_pred, y_test)

print("結果")
print("single:{:e}, ensemble:{:e}".format(min(mse_single), mse))

結果
single:2.967268e+09, ensemble:2.742822e+09


In [7]:
# パターン2
mse_single = []
lr_2 = LinearRegression()
lr_2.fit(X_train, y_train)
y_pred = lr_2.predict(X_test)
mse_single.append(mean_squared_error(y_pred, y_test))

lr_3 = LinearRegression()
lr_3.fit(X_train, y_train)
y_pred = lr_3.predict(X_test)
mse_single.append(mean_squared_error(y_pred, y_test))

dt_2 = DecisionTreeRegressor(max_depth=3)
dt_2.fit(X_train, y_train)
y_pred = dt_2.predict(X_test)
mse_single.append(mean_squared_error(y_pred, y_test))

svr_2 = SVR(C=0.1)
svr_2.fit(X_train, y_train)
y_pred = svr_2.predict(X_test)
mse_single.append(mean_squared_error(y_pred, y_test))


models_2 = [lr_2, lr_3, dt_2, svr_2]

blend_model = ScratchBlending(models_2)
y_pred = blend_model.predict(X_test)
mse = mean_squared_error(y_pred, y_test)

print("結果")
print("single:{:e}, ensemble:{:e}".format(min(mse_single), mse))

結果
single:2.879109e+09, ensemble:2.810552e+09


In [8]:
# パターン3
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

X_train_scale = scaler.fit_transform(X_train)
X_test_scale = scaler.transform(X_test)

mse_single = []
lr_4 = LinearRegression()
lr_4.fit(X_train_scale, y_train)
y_pred = lr_4.predict(X_test_scale)
mse_single.append(mean_squared_error(y_pred, y_test))

svr_3 = SVR()
svr_3.fit(X_train_scale, y_train)
y_pred = svr_3.predict(X_test_scale)
mse_single.append(mean_squared_error(y_pred, y_test))

dt_3 = DecisionTreeRegressor()
dt_3.fit(X_train_scale, y_train)
y_pred = dt_3.predict(X_test_scale)
mse_single.append(mean_squared_error(y_pred, y_test))


models_3 = [lr_4, dt_3, svr_3]

blend_model = ScratchBlending(models_3)
y_pred = blend_model.predict(X_test_scale)
mse = mean_squared_error(y_pred, y_test)

print("結果")
print("single:{:e}, ensemble:{:e}".format(min(mse_single), mse))

結果
single:2.967268e+09, ensemble:2.600892e+09


### 【問題2】バギングのスクラッチ実装
バギング をスクラッチ実装し、単一モデルより精度があがる例を 最低1つ 示してください。


#### バギングとは
バギングは入力データの選び方を多様化する方法です。学習データから重複を許した上でランダムに抜き出すことで、N種類のサブセット（ ブートストラップサンプル ）を作り出します。それらによってモデルをN個学習し、推定結果の平均をとります。ブレンディングと異なり、それぞれの重み付けを変えることはありません。

[sklearn.model_selection.train_test_split — scikit-learn 0.21.3 documentation](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html)

scikit-learnのtrain_test_splitを、shuffleパラメータをTrueにして使うことで、ランダムにデータを分割することができます。これによりブートストラップサンプルが手に入ります。

推定結果の平均をとる部分はブースティングと同様の実装になります。

In [9]:
models_4 = []
mse_single = []

num_tree = 10
depth_tree = 10

for i in range(num_tree):
    X_train_bag, X_test_bag, y_train_bag, y_test_bag = train_test_split(X_train, y_train, 
                                                                        shuffle=True,
                                                                        test_size=0.2)
    dt_tmp = DecisionTreeRegressor(max_depth=depth_tree)
    dt_tmp.fit(X_train_bag, y_train_bag)
    y_pred = dt_tmp.predict(X_test)
    mse_single.append(mean_squared_error(y_pred, y_test))
    models_4.append(dt_tmp)
    
bagging_model = ScratchBlending(models_4)
y_pred = bagging_model.predict(X_test)
mse = mean_squared_error(y_pred, y_test)

print("結果")
print("single:{:e}, ensemble:{:e}".format(min(mse_single), mse))

結果
single:2.036735e+09, ensemble:2.077508e+09


### 【問題3】スタッキングのスクラッチ実装
スタッキング をスクラッチ実装し、単一モデルより精度があがる例を 最低1つ 示してください。

#### スタッキングとは
スタッキングの手順は以下の通りです。最低限ステージ0とステージ1があればスタッキングは成立するため、それを実装してください。まずは $K_0=3, M_0=2$ 程度にします。

《学習時》

（ステージ $0$ ）

- 学習データを $K_0$ 個に分割する。
- 分割した内の $(K_0 - 1)$ 個をまとめて学習用データ、残り $1$ 個を推定用データとする組み合わせが $K_0$ 個作れる。
- あるモデルのインスタンスを $K_0$ 個用意し、異なる学習用データを使い学習する。
- それぞれの学習済みモデルに対して、使っていない残り $1$ 個の推定用データを入力し、推定値を得る。（これをブレンドデータと呼ぶ）
- さらに、異なるモデルのインスタンスも $K_0$ 個用意し、同様のことを行う。モデルが $M_0$ 個あれば、 $M_0$ 個のブレンドデータが得られる。

（ステージ $n$ ）

ステージ $n-1$ のブレンドデータを$M_{n-1}$ 次元の特徴量を持つ学習用データと考え、 $K_n$ 個に分割する。以下同様である。

（ステージ $N$ ）＊最後のステージ

ステージ $N-1$ の $M_{N-1}$ 個のブレンドデータを$M_{N-1}$ 次元の特徴量の入力として、1種類のモデルの学習を行う。これが最終的な推定を行うモデルとなる。

《推定時》

（ステージ $0$ ）

テストデータを $K_0×M_0$ 個の学習済みモデルに入力し、$K_0×M_0$ 個の推定値を得る。これを $K_0$ の軸で平均値を求め $M_0$ 次元の特徴量を持つデータを得る。（ブレンドテストと呼ぶ）

（ステージ $n$ ）

ステージ $n-1$ で得たブレンドテストを $K_n×M_n$ 個の学習済みモデルに入力し、$K_n×M_n$ 個の推定値を得る。これを $K_n$ の軸で平均値を求め $M_0$ 次元の特徴量を持つデータを得る。（ブレンドテストと呼ぶ）

（ステージ $N$ ）＊最後のステージ

ステージ $N-1$ で得たブレンドテストを学習済みモデルに入力し、推定値を得る。

In [10]:
import numpy as np
import itertools
import copy
from sklearn.model_selection import KFold

In [11]:
kf = KFold(n_splits=3, shuffle=False)
a = np.array([[1,2,3,4,5,6], [2,3,4,5,6,7], [3,4,5,6,7,8], [1,2,3,4,5,6], [2,3,4,5,6,7], [3,4,5,6,7,8], [1,2,3,4,5,6], [2,3,4,5,6,7], [3,4,5,6,7,8]])
index = list(kf.split(a))
index

[(array([3, 4, 5, 6, 7, 8]), array([0, 1, 2])),
 (array([0, 1, 2, 6, 7, 8]), array([3, 4, 5])),
 (array([0, 1, 2, 3, 4, 5]), array([6, 7, 8]))]

In [19]:
class ScratchStacking():
    def __init__(self, models, K):
        self.K = K
        self.models = models
        self.M = len(models)
        self.last_model = None
        self.stack_models = []
        self.single_mse = {}
        self.flg_reshape = 0

    def fit(self, X, y):
        X = np.array(X)
        y = np.array(y)
        kf = KFold(n_splits=self.K, shuffle=False)
        
        # ステージ0
        for m in range(self.M):
            tmp_stack_models = []
            self.single_mse["model_"+str(m)] = []
            for k, (train_index, val_index) in enumerate(kf.split(X)):
                # 学習・推定
                self.models[m].fit(X[train_index], y[train_index])
                pred_tmp = self.models[m].predict(X[val_index])
                if k == 0:
                    tmp_blend_data = pred_tmp
                    y_test = y[val_index]
                else:
                    tmp_blend_data = np.hstack([tmp_blend_data, pred_tmp])
                    y_test = np.hstack([y_test, y[val_index]])

                self.single_mse["model_"+str(m)].append(mean_squared_error(pred_tmp, y[val_index]))
                    
                tmp_stack_models.append(copy.deepcopy(self.models[m]))

            self.stack_models.append(tmp_stack_models)

            if m == 0:
                blend_data = tmp_blend_data
            else:
                blend_data = np.vstack([blend_data, tmp_blend_data])

        # 最終ステージ
        blend_data = blend_data.T
        self.last_model = RandomForestRegressor(max_depth=5)
        if blend_data.shape == y_test.shape:
            self.flg_reshape = 1
            blend_data = blend_data.reshape(-1, 1)
        self.last_model.fit(blend_data, y_test)

    def predict(self, X):
        # ステージ0
        for m in range(self.M):
            for k in range(self.K):
                if k == 0:
                    y_pred = self.stack_models[m][k].predict(X)
                else:
                    y_pred = np.vstack([y_pred, self.stack_models[m][k].predict(X)])
            
            y_pred_mean = y_pred.T.mean(axis=1)
            
            if m == 0:
                y_pred_mean_all = y_pred_mean
            else:
                y_pred_mean_all = np.vstack([y_pred_mean_all, y_pred_mean])
        
        y_pred_mean_all = y_pred_mean_all.T

        # 最終ステージ
        if self.flg_reshape == 1:
            y_pred_mean_all = y_pred_mean_all.reshape(-1, 1)
        result = self.last_model.predict(y_pred_mean_all)
        
        return result

In [16]:
lr_5 = LinearRegression()
svr_4 = SVR()
dt_4 = DecisionTreeRegressor(max_depth=10)
rdg = Ridge()
lasso = Lasso()
# last_model = DecisionTreeRegressor()
models_4 = [dt_4, lr_5, rdg, lasso]

stacking = ScratchStacking(models_4, 3)
stacking.fit(X_train, y_train)
pred = stacking.predict(X_test)
mse = mean_squared_error(pred, y_test)

print(stacking.single_mse)
print("結果")
print("single:{:e}, ensemble:{:e}".format(min(min(stacking.single_mse.values())), mse))

{'model_0': [2320375826.2049193, 2623001474.1875887, 2134598984.620627], 'model_1': [1774895498.8549366, 2481788907.223859, 1773550877.2380145], 'model_2': [1774895303.6456006, 2481789050.5483913, 1773550927.4333797], 'model_3': [1774895354.2016983, 2481789018.9762273, 1773550907.4525685]}
結果
single:1.773551e+09, ensemble:3.270910e+09


In [18]:
logreg = LogisticRegression(max_iter=5000)
rdg = Ridge()
lasso = Lasso()
eln = ElasticNet()
lr = LinearRegression()
# last_model = LogisticRegression(max_iter=5000)
models_5 = [logreg, rdg, lasso, eln, lr]

stacking = ScratchStacking(models_5, 3)
stacking.fit(X_train, y_train)
pred = stacking.predict(X_test)
mse = mean_squared_error(pred, y_test)

print(stacking.single_mse)
print("結果")
print("single:{:e}, ensemble:{:e}".format(min(min(stacking.single_mse.values())), mse))

{'model_0': [3389068602.928205, 4155719280.359897, 3833670714.9717226], 'model_1': [1774895303.6456006, 2481789050.5483913, 1773550927.4333797], 'model_2': [1774895354.2016983, 2481789018.9762273, 1773550907.4525685], 'model_3': [1774819832.3176727, 2481845086.442278, 1773570715.4008763], 'model_4': [1774895498.8549366, 2481788907.223859, 1773550877.2380145]}
結果
single:1.773571e+09, ensemble:1.002327e+10


In [20]:
eln = ElasticNet()
dt = DecisionTreeRegressor(max_depth=10)
svr = SVR(kernel='poly')
lr = LinearRegression()
rdg = Ridge()
lasso = Lasso()
logreg = LogisticRegression(max_iter=5000)
# last_model = RandomForestRegressor(max_depth=5)

models_6 = [eln, dt, svr, lr, rdg, lasso, logreg]

stacking = ScratchStacking(models_6, 3)
stacking.fit(X_train, y_train)
pred = stacking.predict(X_test)
mse = mean_squared_error(pred, y_test)

print(stacking.single_mse)
print("結果")
print("single:{:e}, ensemble:{:e}".format(min(min(stacking.single_mse.values())), mse))

{'model_0': [1774819832.3176727, 2481845086.442278, 1773570715.4008763], 'model_1': [2317359570.841956, 2639037838.717794, 2176177872.795434], 'model_2': [2483578611.9407163, 3768169884.61499, 2394386522.9104295], 'model_3': [1774895498.8549366, 2481788907.223859, 1773550877.2380145], 'model_4': [1774895303.6456006, 2481789050.5483913, 1773550927.4333797], 'model_5': [1774895354.2016983, 2481789018.9762273, 1773550907.4525685], 'model_6': [3389068602.928205, 4155719280.359897, 3833670714.9717226]}
結果
single:1.773571e+09, ensemble:1.991638e+09


In [21]:
eln = ElasticNet(alpha=0.4)
dt = DecisionTreeRegressor(max_depth=7)
svr = SVR(kernel='poly', C=0.2)
lr = LinearRegression()
rdg = Ridge(alpha=0.7)
lasso = Lasso(alpha=0.3)
# last_model = RandomForestRegressor(max_depth=5)

models_7 = [eln, dt, svr, lr, rdg, lasso]

stacking = ScratchStacking(models_7, 3)
stacking.fit(X_train, y_train)
pred = stacking.predict(X_test)
mse = mean_squared_error(pred, y_test)

print(stacking.single_mse)
print("結果")
print("single:{:e}, ensemble:{:e}".format(min(min(stacking.single_mse.values())), mse))

{'model_0': [1774865150.7670772, 2481811309.1378307, 1773558748.061693], 'model_1': [1923577328.2195165, 2326062530.667825, 1566711232.7192554], 'model_2': [2839004356.282099, 4140245011.845541, 2606565876.599297], 'model_3': [1774895498.8549366, 2481788907.223859, 1773550877.2380145], 'model_4': [1774895362.207922, 2481789007.5506124, 1773550912.3743963], 'model_5': [1774895456.3711839, 2481788942.930067, 1773550887.241639]}
結果
single:1.773559e+09, ensemble:1.761802e+09
